In [1]:
import pickle
import numpy as np
import pandas as pd

In [45]:
with open("raw_sensor_data.pkl", "rb") as f:
    sensor_data = pickle.load(f)

# with open("imputed_sensor_data.pkl", "rb") as f:
#     sensor_data = pickle.load(f)

In [3]:
def build_comparison_df(sensor_data, mat, sensor):
    data_lengths = []
    for heater_idx in sensor_data[f"mat_{mat}"][sensor]:
        heater_data_len = sensor_data[f"mat_{
            mat}"][sensor][heater_idx].shape[0]
        data_lengths.append(heater_data_len)

    min_len = min(data_lengths)
    heater_dict = {}
    for heater_idx in sensor_data[f"mat_{mat}"][sensor]:
        heater_dates = sensor_data[f"mat_{
            mat}"][sensor][heater_idx]["Date"].to_numpy()
        heater_data_len = len(heater_dates)
        if heater_data_len > min_len:
            heater_dates = heater_dates[:min_len]
        heater_dict[heater_idx] = heater_dates

    for heater_idx in range(1, 10):
        diff_seconds = (heater_dict[0] - heater_dict[heater_idx]
                        ).astype("timedelta64[s]").astype(np.int32)
        heater_dict[f"d 0-{heater_idx}"] = diff_seconds

    df = pd.DataFrame(heater_dict)
    return df

In [4]:
def impute_data_mean(df, after):
    df_copy = df.copy()
    line = df_copy.iloc[after:after+2].mean()

    line["Sensor Index"] = int(line["Sensor Index"])
    line["Sensor ID"] = int(line["Sensor ID"])
    line["Time Since PowerOn"] = int(line["Time Since PowerOn"])
    line["Real time clock"] = int(line["Real time clock"])
    line["Heater Profile Step Index"] = int(line["Heater Profile Step Index"])
    line["Scanning Mode Enabled"] = int(line["Scanning Mode Enabled"])
    line["Scanning Cycle Index"] = int(line["Scanning Cycle Index"])
    line["Label Tag"] = int(line["Label Tag"])
    line["Error Code"] = int(line["Error Code"])
    line["Date"] = line["Date"].floor("S")

    index = df_copy.iloc[after:after+2].index
    index = int((index[0] + index[1]) / 2)
    df_copy.loc[index] = line
    df_copy.sort_index(inplace=True)
    return df_copy

In [39]:
MATRIX = 1

In [71]:
for i in range(0, 8):
    df = build_comparison_df(sensor_data, MATRIX, i)
    df.to_excel(f"mat_{MATRIX}_s{i}_diff.xlsx")
    break

In [30]:
sensor_data[f"mat_{MATRIX}"][0][8].iloc[6972:6976]

,Sensor Index,Sensor ID,Time Since PowerOn,Real time clock,Temperature,Pressure,Relative Humidity,Resistance Gassensor,Heater Profile Step Index,Scanning Mode Enabled,Scanning Cycle Index,Label Tag,Error Code,Date
303337,0,355892538,75581257,1729766537,37.944469,933.719727,18.580612,8354921.0,8,1,1,0,0,2024-10-24 13:42:17
303379,0,355892538,75592054,1729766548,38.034630,933.747925,18.523739,8329435.5,8,1,1,0,0,2024-10-24 13:42:28
303455,0,355892538,75613874,1729766570,37.899391,933.752136,18.551878,8432321.0,8,1,1,0,0,2024-10-24 13:42:50
303500,0,355892538,75624639,1729766581,37.936958,933.721619,18.554167,8393443.0,8,1,1,0,0,2024-10-24 13:43:01


In [69]:
sensor = 7
heater = 7
after = 2036
df_copy = impute_data_mean(sensor_data[f"mat_{MATRIX}"][sensor][heater], after)
sensor_data[f"mat_{MATRIX}"][sensor][heater] = df_copy.copy()
df_copy.iloc[after-1:after+4]

,Sensor Index,Sensor ID,Time Since PowerOn,Real time clock,Temperature,Pressure,Relative Humidity,Resistance Gassensor,Heater Profile Step Index,Scanning Mode Enabled,Scanning Cycle Index,Label Tag,Error Code,Date
155055,7,355883595,38632193,1729729548,34.199619,934.522217,18.869629,2947823.00,7,1,1,0,0,2024-10-24 03:25:48
155126,7,355883595,38651288,1729729567,34.139702,934.529053,18.902536,2922583.00,7,1,1,0,0,2024-10-24 03:26:07
155202,7,355883595,38670236,1729729586,34.217092,934.511383,18.852424,2935203.00,7,1,1,0,0,2024-10-24 03:26:26
155279,7,355883595,38689184,1729729605,34.294483,934.493713,18.802311,2947823.00,7,1,1,0,0,2024-10-24 03:26:45
155359,7,355883595,38708138,1729729624,34.244553,934.497314,18.788864,2910124.25,7,1,1,0,0,2024-10-24 03:27:04


In [72]:
with open("imputed_sensor_data.pkl", "wb") as f:
    pickle.dump(sensor_data, f)

with open("imputed_sensor_data.pkl", "rb") as f:
    sensor_data = pickle.load(f)